# Labeled Point Data

In this notebook we're going to implement a custom tile generator to serve custome data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import h5py
import numpy as np
import pandas as pd

### Load data

In this example we show a dataset on image-based spatial transcriptomics by [Wang et al., 2018.](https://doi.org/10.1038/s41598-018-22297-7)

In [3]:
df = pd.read_csv('data/spatial_gene_expression.csv.gz')

print('Number of points: {}'.format(len(df)))
df.head()

Number of points: 2601157


,x,y,gene
0,791.8022,229.1912,ENO1
1,901.6493,897.6785,MTHFD1
2,708.8124,902.3960,THBS1
3,-690.2937,-870.6509,PPM1G
4,-52.4658,-278.9252,ENO1


### Custom tileset API

To generate and serve custom tiles we need a `tileset_info()` and a `tiles()` function as demonstrated below.

In [4]:
from higlass import Tileset
from clodius.tiles.format import format_dense_tile
from clodius.tiles.utils import tile_bounds

def dfdensity(df, channel_col=None, channel=None, uuid=None, max_zoom=5, **kwargs):
    
    # [Optional] Subsample data
    if channel_col and channel:
        data = df[df[channel_col] == channel].reindex(columns=['x', 'y']).values
    else:
        data = df.reindex(columns=['x', 'y']).values

    tileset_info = {
        'min_pos': [df['x'].min(), df['y'].min()],
        'max_pos': [df['x'].max(), df['y'].max()],
        'max_width': max(
            df['x'].max() - df['x'].min(),
            df['y'].max() - df['y'].min()
        ),
        'max_zoom': max_zoom,
        'mirror_tiles': 'false'
    }
            
    def get_tile(z, x, y):
        extent = tile_bounds(tileset_info, z, x, y)
        
        # get all the points within the extent
        points = data[
            (data[:, 0] > extent[0]) &
            (data[:, 0] < extent[2]) &
            (data[:, 1] > extent[1]) &
            (data[:, 1] < extent[3])
        ]
        
        # Generate a 2D histogram
        hist, _, _ = np.histogram2d(
            points[:, 0],
            points[:, 1],
            bins=256
        )
        
        # Set empty bins to `nan` to make them transparent
        hist[hist == 0.] = np.nan
        
        return hist.T
    
    def tiles(tile_ids):
        tiles = []
        
        for tile_id in tile_ids:
            _, z, x, y = tile_id.split('.')
            # `format_dense_tile()` converts the ndarray to base64 encoding
            # and adds min/max values
            data = format_dense_tile(get_tile(int(z), int(x), int(y)))
            tiles.append((tile_id, data))
    
        return tiles

    return Tileset(
        uuid=uuid,
        tileset_info=lambda: tileset_info,
        tiles=tiles,
        **kwargs
    )

Let's visualize the data as usual

In [5]:
from higlass import Track, View, display

all_genes_tileset = dfdensity(df, uuid='density', name='Wang et al., 2018. All Genes')

track_config = {
    'track_type': 'heatmap',
    'position': 'center',
    'height': 600,
    'options': {
        'colorRange': ['#ffbb33', '#e5001c', 'black'],
        'backgroundColor': 'white',
    }
}

widget, server, viewconf = display([
    View([
        Track(tileset=all_genes_tileset, **track_config),
    ]),
])

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'P04VyuHVQvCCL…

Because we implemented a custom tileset API we can add any kind of customization like picking which channels (i.e., genes) we actually want to visualize.

In [6]:
## Important change ##########
eno1_tileset = dfdensity(
    df, channel_col='gene', channel='ENO1', uuid='density_eno1',
    name='Wang et al., 2018. ENO1 Only'
)
notch2_tileset = dfdensity(
    df, channel_col='gene', channel='NOTCH2', uuid='density_rad21',
    name='Wang et al., 2018. NOTCH2 Only'
)
##############################

view_config = {
    'y': 0,
    'width': 4,
}

all_genes_track = Track(tileset=all_genes_tileset, **track_config)
eno1_track = Track(tileset=eno1_tileset, **track_config)
notch2_track = Track(tileset=notch2_tileset, **track_config)

widget, server, viewconf = display([
    View(x=0, tracks=[all_genes_track], **view_config),
    View(x=4, tracks=[eno1_track], **view_config),
    View(x=8, tracks=[notch2_track], **view_config),
])

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'LREtaRqlQhCFC…

**Plasma vs nuclear transcribed RNAs**

In the following closeup we can see that ENO1 is primarily transcriped in the cell body (outside the nucleolus) while NOTCH2 is transcribed within the nucleolus)

In [7]:
from copy import deepcopy

close_up_view_config = deepcopy(view_config)
close_up_view_config['initialXDomain'] = [-333, -209]
close_up_view_config['initialYDomain'] = [-202, -30]

closeup_all_genes_track = all_genes_track.change_options(maxZoom=2)
closeup_eno1_track = eno1_track.change_options(maxZoom=2)
closeup_notch2_track = notch2_track.change_options(maxZoom=2)

widget, server, viewconf = display([
    View(x=0, tracks=[closeup_all_genes_track], **close_up_view_config),
    View(x=4, tracks=[closeup_eno1_track], **close_up_view_config),
    View(x=8, tracks=[closeup_notch2_track], **close_up_view_config),
])

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'CV0-QJ7DTLuhe…

### Tileset Response

Most HiGlass tracks expect the data to be base64 encoded numpy arrays (`dense`) with a the data type (`dtype`), length `size`, and precalculated min and max values (`min_value` and `max_value`) for efficient value scaling.

In [8]:
response = server.tiles(all_genes_tileset.uuid, 0, 0, 0)
print(
    f'dtype: {response["dtype"]}', 
    f'tile_size: {response["size"]}',
    f'min_value: {response["min_value"]}' ,
    f'max_value: {response["max_value"]}' ,
    f'dense: {response["dense"][:20]}...',
    sep='\n'
)

dtype: float32
tile_size: 256
min_value: 1.0
max_value: 369.0
dense: AABwQgAANEIAAJBBAAAA...


### View config

To ultimately visualize the data, HiGlass' viewer requires a view configuration telling HiGlass the what, how, and where.

In [9]:
viewconf

{
  "editable": true,
  "views": [
    {
      "uid": "CV0-QJ7DTLuheuGyXv7WzQ",
      "tracks": {
        "top": [],
        "center": [
          {
            "type": "heatmap",
            "options": {
              "colorRange": [
                "#ffbb33",
                "#e5001c",
                "black"
              ],
              "backgroundColor": "white",
              "maxZoom": 2
            },
            "tilesetUid": "density",
            "height": 600,
            "server": "http://localhost:44141/api/v1"
          }
        ],
        "left": [],
        "right": [],
        "bottom": []
      },
      "layout": {
        "w": 4,
        "h": 6,
        "x": 0,
        "y": 0
      },
      "initialXDomain": [
        -333,
        -209
      ],
      "initialYDomain": [
        -202,
        -30
      ]
    },
    {
      "uid": "WjD10TLITAy3l2KlV41sFQ",
      "tracks": {
        "top": [],
        "center": [
          {
            "type": "heatmap",
          

☝️ This dictionary ultimately tells the JavaScript library what dataset to visualize where and how.